In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.models import load_model

from adversarialdefence.utils import ModelUtils, GeneralUtils

### ***Data Loading***

In [2]:
df = pd.read_csv('../csv/CICIDS2017_improved-preprocessed.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df.shape

(1715326, 49)

In [3]:
df_benign = df[df['Label'] == 0]
df_anomalous = df[df['Label'] == 1]

print(f'Number of benign samples: {df_benign.shape[0]}')
print(f'Number of anomalous samples: {df_anomalous.shape[0]}')

Number of benign samples: 1432918
Number of anomalous samples: 282408


### ***Load models and scalers***

In [4]:
autoencoder = load_model('../modelli/autoencoder_best_weights_96-96.hdf5')
dnn = load_model('../modelli/DNN_best_weights_99.hdf5')

In [5]:
std_scaler_aut = joblib.load('../modelli/std_scaler_aut.bin')
std_scaler_dnn = joblib.load('../modelli/std_scaler_dnn.bin')
columns = df.copy().drop('Label', axis=1).columns

/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### ***Adversarial WB samples loading***

In [6]:
base_advs_csv_path = '../csv/'

df_fgsm, X_fgsm, y_fgsm = GeneralUtils.get_data_with_advs(base_advs_csv_path + 'fgsm.csv')
df_deepFool, X_deepFool, y_deepFool = GeneralUtils.get_data_with_advs(base_advs_csv_path + 'deepFool.csv')
df_carlini, X_carlini, y_carlini = GeneralUtils.get_data_with_advs(base_advs_csv_path + 'carlini.csv')

X_carlini.drop(X_carlini.columns[0], axis=1, inplace=True)
df_carlini.drop(df_carlini.columns[0], axis=1, inplace=True)

In [7]:
preds_fgsm_dnn = ModelUtils.binary_preds_supervised(dnn, X_fgsm)
preds_deepFool_dnn = ModelUtils.binary_preds_supervised(dnn, X_deepFool)
preds_carlini_dnn = ModelUtils.binary_preds_supervised(dnn, X_carlini)

preds_fgsm_aut = ModelUtils.binary_preds_unsupervised(autoencoder, X_fgsm)
preds_deepFool_aut = ModelUtils.binary_preds_unsupervised(autoencoder, X_deepFool)
preds_carlini_aut = ModelUtils.binary_preds_unsupervised(autoencoder, X_carlini)

In [8]:
advs_fgsm_dnn = GeneralUtils.get_advs_samples(preds_fgsm_dnn, df_fgsm)
advs_deepFool_dnn = GeneralUtils.get_advs_samples(preds_deepFool_dnn, df_deepFool)
advs_carlini_dnn = GeneralUtils.get_advs_samples(preds_carlini_dnn, df_carlini)

advs_fgsm_aut = GeneralUtils.get_advs_samples(preds_fgsm_aut, df_fgsm)
advs_deepFool_aut = GeneralUtils.get_advs_samples(preds_deepFool_aut, df_deepFool)
advs_carlini_aut = GeneralUtils.get_advs_samples(preds_carlini_aut, df_carlini)

In [9]:
df_advs_dnn = pd.concat([advs_fgsm_dnn, advs_deepFool_dnn, advs_carlini_dnn], ignore_index=True).sample(frac=1)
df_advs_aut = pd.concat([advs_fgsm_aut, advs_deepFool_aut, advs_carlini_aut], ignore_index=True).sample(frac=1)

print(f'Number of adversarial samples for dnn: {df_advs_dnn.shape[0]}')
print(f'Number of adversarial samples for aut: {df_advs_aut.shape[0]}')

Number of adversarial samples for dnn: 358351
Number of adversarial samples for aut: 270813


### ***Data splitting***

In [10]:
def data_splitting_AUT_with_advs(df_benign, df_anomalous, df_advs):
    
    df_benign = pd.DataFrame(std_scaler_aut.transform(df_benign.sample(frac=0.15).drop('Label', axis=1)), columns=columns).assign(Label=0)
    df_anomalous = pd.DataFrame(std_scaler_aut.transform(df_anomalous.copy().drop('Label', axis=1)), columns=columns).assign(Label=1)
            
    df_test = pd.concat([df_benign, df_anomalous], ignore_index=True).sample(frac=1.0)

    print(f'Anomalous samples test AUT: {df_anomalous.shape[0]}')
    print(f'Benign samples test AUT: {df_benign.shape[0]}')
    print(f'Adversarial samples test AUT: {df_advs.shape[0]}')
    print()
    
    return df_test

def data_splitting_DNN_with_advs(df_benign, df_anomalous, df_advs):
    df_benign = pd.DataFrame(std_scaler_dnn.transform(df_benign.copy().drop('Label', axis=1)), columns=columns).assign(Label=0)
    df_anomalous = pd.DataFrame(std_scaler_dnn.transform(df_anomalous.copy().drop('Label', axis=1)), columns=columns).assign(Label=1)
    
    df_anomalous_train, df_anomalous_test = train_test_split(df_anomalous, test_size=0.25, random_state=42)
    df_anomalous_train, df_anomalous_val = train_test_split(df_anomalous_train, test_size=0.15, random_state=42)

    df_benign_train, df_benign_test = train_test_split(df_benign, test_size=0.10, random_state=42)
    df_benign_train, df_benign_val = train_test_split(df_benign_train, test_size=0.10, random_state=42)
    
    df_advs_train, df_advs_test = train_test_split(df_advs, test_size=0.20, random_state=42)
    df_advs_train, df_advs_val = train_test_split(df_advs_train, test_size=0.20, random_state=42)

    df_benign_train = df_benign_train.sample(frac=0.40)

    print(f'Anomalous samples train/val/test DNN: {df_anomalous_train.shape[0]} - {df_anomalous_val.shape[0]} - {df_anomalous_test.shape[0]}')
    print(f'Benign samples train/val/test DNN: {df_benign_train.shape[0]} - {df_benign_val.shape[0]} - {df_benign_test.shape[0]}')
    print(f'Adversarial samples train/val/test DNN: {df_advs_train.shape[0]} - {df_advs_val.shape[0]} - {df_advs_test.shape[0]}')
    print()
    
    df_train = pd.concat([df_benign_train, df_anomalous_train, df_advs_train], ignore_index=True).sample(frac=1)
    df_test = pd.concat([df_benign_test, df_anomalous_test, df_advs_test], ignore_index=True).sample(frac=1)
    df_val = pd.concat([df_benign_val, df_anomalous_val, df_advs_val], ignore_index=True).sample(frac=1)
    
    return df_train, df_val, df_test

In [11]:
df_test_aut = data_splitting_AUT_with_advs(df_benign, df_anomalous, df_advs_aut)
df_train_dnn, df_val_dnn, df_test_dnn = data_splitting_DNN_with_advs(df_benign, df_anomalous, df_advs_dnn)

print(f'Number of train/val/test samples for dnn: {df_train_dnn.shape[0]} - {df_val_dnn.shape[0]} - {df_test_dnn.shape[0]}')
print(f'Number of test samples for aut: {df_test_aut.shape[0]}')

Anomalous samples test AUT: 282408
Benign samples test AUT: 214938
Adversarial samples test AUT: 270813

Anomalous samples train/val/test DNN: 180035 - 31771 - 70602
Benign samples train/val/test DNN: 464265 - 128963 - 143292
Adversarial samples train/val/test DNN: 229344 - 57336 - 71671

Number of train/val/test samples for dnn: 873644 - 218070 - 285565
Number of test samples for aut: 497346


In [12]:
#Training datasets
X_train_dnn = df_train_dnn.copy()
y_train_dnn = X_train_dnn.pop('Label')

#Validation datasets
X_val_dnn = df_val_dnn.copy()
y_val_dnn = X_val_dnn.pop('Label')

#Test datasets
X_test_aut = df_test_aut.copy()
y_test_aut = X_test_aut.pop('Label')

X_test_dnn = df_test_dnn.copy()
y_test_dnn = X_test_dnn.pop('Label')

### ***Aversarial retraining***

In [13]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=10,
    verbose=1, 
    mode='min',
    restore_best_weights=True
)

save_model = tf.keras.callbacks.ModelCheckpoint(
    filepath='dnn_retrained_best_weights.hdf5',
    save_best_only=True,
    monitor='val_loss',
    verbose=0,
    mode='min'
)

In [24]:
BATCH_SIZE = 1024
EPOCHS = 10

history = dnn.fit(
    X_train_dnn, y_train_dnn,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[early_stop, save_model],
    validation_data=(X_val_dnn, y_val_dnn)
)

Epoch 1/10
854/854 [==============================] - 2s 2ms/step - loss: 0.1264 - acc: 0.9696 - val_loss: 0.0431 - val_acc: 0.9934
Epoch 2/10
128/854 [===>..........................] - ETA: 0s - loss: 0.0436 - acc: 0.9927

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


854/854 [==============================] - 1s 1ms/step - loss: 0.0413 - acc: 0.9929 - val_loss: 0.0378 - val_acc: 0.9936
Epoch 3/10
854/854 [==============================] - 1s 1ms/step - loss: 0.0385 - acc: 0.9931 - val_loss: 0.0324 - val_acc: 0.9946
Epoch 4/10
854/854 [==============================] - 1s 1ms/step - loss: 0.0281 - acc: 0.9953 - val_loss: 0.0240 - val_acc: 0.9962
Epoch 5/10
854/854 [==============================] - 1s 2ms/step - loss: 0.0251 - acc: 0.9958 - val_loss: 0.0242 - val_acc: 0.9960
Epoch 6/10
854/854 [==============================] - 1s 1ms/step - loss: 0.0249 - acc: 0.9958 - val_loss: 0.0239 - val_acc: 0.9961
Epoch 7/10
854/854 [==============================] - 2s 2ms/step - loss: 0.0249 - acc: 0.9958 - val_loss: 0.0236 - val_acc: 0.9961
Epoch 8/10
854/854 [==============================] - 1s 2ms/step - loss: 0.0249 - acc: 0.9958 - val_loss: 0.0231 - val_acc: 0.9962
Epoch 9/10
854/854 [==============================] - 1s 1ms/step - loss: 0.0245 - acc:

In [10]:
dnn_retrained = load_model('../modelli/dnn_retrained_best_weights.hdf5')

### ***Testing on WB Adversarial samples***

In [11]:
BASE_ADVS_CSV_PATH = '../csv/'

In [26]:
def checkAdversarialResults(X_advs, df_advs, attack_type, num_total_attempts, dnn_target, aut_binary_th=0.05, context=''):
    preds_advs_dnn = ModelUtils.binary_preds_supervised(dnn_target, X_advs)
    advs_samples_dnn = GeneralUtils.get_advs_samples(np.array(preds_advs_dnn), df_advs)
    print(f'Successful adversarial samples of type {attack_type} for DNN {context}: {advs_samples_dnn.shape[0]} / {num_total_attempts} - ASR: {(advs_samples_dnn.shape[0]/num_total_attempts) * 100}%')

    preds_advs_aut = ModelUtils.binary_preds_unsupervised(autoencoder, X_advs, aut_binary_th)
    advs_samples_aut = GeneralUtils.get_advs_samples(np.array(preds_advs_aut), df_advs)
    print(f'Successful adversarial samples of type {attack_type} for AUT {context}: {advs_samples_aut.shape[0]} / {num_total_attempts} - ASR: {(advs_samples_aut.shape[0]/num_total_attempts) * 100}%')

    preds_advs_dnn_by_aut = ModelUtils.binary_preds_unsupervised(autoencoder, advs_samples_dnn.copy().drop('Label', axis=1), aut_binary_th)
    common_advs_samples = GeneralUtils.get_advs_samples(np.array(preds_advs_dnn_by_aut), advs_samples_dnn)
    print(f'Successful adversarial samples of type {attack_type} for TARGET SYSTEM {context}: {common_advs_samples.shape[0]} / {num_total_attempts} - ATSR: {(common_advs_samples.shape[0]/num_total_attempts) * 100}%')

**FGSM**

In [27]:
checkAdversarialResults(X_fgsm, df_fgsm, 'FGSM', df_anomalous.shape[0], dnn)
print('---------')
checkAdversarialResults(X_fgsm, df_fgsm, 'FGSM', df_anomalous.shape[0], dnn_retrained, 0.035, 'RETRAINED')

Successful adversarial samples of type FGSM for DNN : 94759 / 282408 - ASR: 33.55393614911759%
Successful adversarial samples of type FGSM for AUT : 16858 / 282408 - ASR: 5.969377638027251%
Successful adversarial samples of type FGSM for TARGET SYSTEM : 4645 / 282408 - ATSR: 1.6447834338970568%
---------
Successful adversarial samples of type FGSM for DNN RETRAINED: 475 / 282408 - ASR: 0.1681963683748336%
Successful adversarial samples of type FGSM for AUT RETRAINED: 1289 / 282408 - ASR: 0.45643182912665364%
Successful adversarial samples of type FGSM for TARGET SYSTEM RETRAINED: 2 / 282408 - ATSR: 0.0007081952352624572%


**DEEPFOOL**

In [28]:
checkAdversarialResults(X_deepFool, df_deepFool, 'DEEPFOOL', df_anomalous.shape[0], dnn)
print('---------')
checkAdversarialResults(X_deepFool, df_deepFool, 'DEEPFOOL', df_anomalous.shape[0], dnn_retrained, 0.035, 'RETRAINED')

Successful adversarial samples of type DEEPFOOL for DNN : 238261 / 282408 - ASR: 84.36765247443415%
Successful adversarial samples of type DEEPFOOL for AUT : 24430 / 282408 - ASR: 8.650604798730914%
Successful adversarial samples of type DEEPFOOL for TARGET SYSTEM : 206 / 282408 - ATSR: 0.0729441092320331%
---------
Successful adversarial samples of type DEEPFOOL for DNN RETRAINED: 1874 / 282408 - ASR: 0.6635789354409224%
Successful adversarial samples of type DEEPFOOL for AUT RETRAINED: 12629 / 282408 - ASR: 4.471898813064786%
Successful adversarial samples of type DEEPFOOL for TARGET SYSTEM RETRAINED: 0 / 282408 - ATSR: 0.0%


**Carlini**

In [29]:
checkAdversarialResults(X_carlini, df_carlini, 'CARLINI', df_anomalous.shape[0], dnn)
print('---------')
checkAdversarialResults(X_carlini, df_carlini, 'CARLINI', df_anomalous.shape[0], dnn_retrained, 0.035, 'RETRAINED')

Successful adversarial samples of type CARLINI for DNN : 25331 / 282408 - ASR: 8.96964675221665%
Successful adversarial samples of type CARLINI for AUT : 229525 / 282408 - ASR: 81.27425568680773%
Successful adversarial samples of type CARLINI for TARGET SYSTEM : 11243 / 282408 - ATSR: 3.9811195150279026%
---------
Successful adversarial samples of type CARLINI for DNN RETRAINED: 467 / 282408 - ASR: 0.16536358743378374%
Successful adversarial samples of type CARLINI for AUT RETRAINED: 143168 / 282408 - ASR: 50.69544772102773%
Successful adversarial samples of type CARLINI for TARGET SYSTEM RETRAINED: 74 / 282408 - ATSR: 0.026203223704710916%


### ***Testing on BB Adversarial samples***

**HopSkipJump**

In [30]:
df_hsj, X_hsj, y_hsj = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'hsj_new.csv')
checkAdversarialResults(X_hsj, df_hsj, 'HOPSKIPJUMP', 50000, dnn)
print('---------')
checkAdversarialResults(X_hsj, df_hsj, 'HOPSKIPJUMP', 50000, dnn_retrained, 0.035, 'RETRAINED')

Successful adversarial samples of type HOPSKIPJUMP for DNN : 49667 / 50000 - ASR: 99.334%
Successful adversarial samples of type HOPSKIPJUMP for AUT : 21511 / 50000 - ASR: 43.022%
Successful adversarial samples of type HOPSKIPJUMP for TARGET SYSTEM : 21500 / 50000 - ATSR: 43.0%
---------
Successful adversarial samples of type HOPSKIPJUMP for DNN RETRAINED: 127 / 50000 - ASR: 0.254%
Successful adversarial samples of type HOPSKIPJUMP for AUT RETRAINED: 5132 / 50000 - ASR: 10.264%
Successful adversarial samples of type HOPSKIPJUMP for TARGET SYSTEM RETRAINED: 7 / 50000 - ATSR: 0.013999999999999999%


**Boundary**

In [31]:
df_boundary, X_boundary, y_boundary = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'boundary_new.csv')
checkAdversarialResults(X_boundary, df_boundary, 'BOUNDARY', 50000, dnn)
print('---------')
checkAdversarialResults(X_boundary, df_boundary, 'BOUNDARY', 50000, dnn_retrained, 0.035, 'RETRAINED')

Successful adversarial samples of type BOUNDARY for DNN : 49847 / 50000 - ASR: 99.694%
Successful adversarial samples of type BOUNDARY for AUT : 191 / 50000 - ASR: 0.382%
Successful adversarial samples of type BOUNDARY for TARGET SYSTEM : 191 / 50000 - ATSR: 0.382%
---------
Successful adversarial samples of type BOUNDARY for DNN RETRAINED: 39597 / 50000 - ASR: 79.194%
Successful adversarial samples of type BOUNDARY for AUT RETRAINED: 124 / 50000 - ASR: 0.248%
Successful adversarial samples of type BOUNDARY for TARGET SYSTEM RETRAINED: 9 / 50000 - ATSR: 0.018000000000000002%


**ZOO**

In [32]:
df_zoo, X_zoo, y_zoo = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'zoo_new.csv')
checkAdversarialResults(X_zoo, df_zoo, 'ZOO', 50000, dnn)
print('---------')
checkAdversarialResults(X_zoo, df_zoo, 'ZOO', 50000, dnn_retrained, 0.035, 'RETRAINED')

Successful adversarial samples of type ZOO for DNN : 3535 / 50000 - ASR: 7.07%
Successful adversarial samples of type ZOO for AUT : 40192 / 50000 - ASR: 80.384%
Successful adversarial samples of type ZOO for TARGET SYSTEM : 978 / 50000 - ATSR: 1.9560000000000002%
---------
Successful adversarial samples of type ZOO for DNN RETRAINED: 80 / 50000 - ASR: 0.16%
Successful adversarial samples of type ZOO for AUT RETRAINED: 25028 / 50000 - ASR: 50.056%
Successful adversarial samples of type ZOO for TARGET SYSTEM RETRAINED: 10 / 50000 - ATSR: 0.02%


**Query Efficient**

In [33]:
df_query_eff_new, X_query_eff_new, y_query_eff_new = GeneralUtils.get_data_with_advs(BASE_ADVS_CSV_PATH + 'query_eff_new.csv')
checkAdversarialResults(X_query_eff_new, df_query_eff_new, 'QUERY EFF', 50000, dnn)
print('---------')
checkAdversarialResults(X_query_eff_new, df_query_eff_new, 'QUERY EFF', 50000, dnn_retrained, 0.035, 'RETRAINED')

Successful adversarial samples of type QUERY EFF for DNN : 24205 / 50000 - ASR: 48.41%
Successful adversarial samples of type QUERY EFF for AUT : 21389 / 50000 - ASR: 42.778%
Successful adversarial samples of type QUERY EFF for TARGET SYSTEM : 37 / 50000 - ATSR: 0.074%
---------
Successful adversarial samples of type QUERY EFF for DNN RETRAINED: 8562 / 50000 - ASR: 17.124%
Successful adversarial samples of type QUERY EFF for AUT RETRAINED: 13471 / 50000 - ASR: 26.942%
Successful adversarial samples of type QUERY EFF for TARGET SYSTEM RETRAINED: 7 / 50000 - ATSR: 0.013999999999999999%


### ***Test Prediction Error on NON Adversarial samples***

In [34]:
X_benign = df_benign.sample(n=100000)
y_benign = X_benign.pop('Label')

X_anomalous = df_anomalous.sample(n=100000)
y_anomalous = X_anomalous.pop('Label')

X_benign_dnn = std_scaler_dnn.transform(X_benign)
X_benign_aut = std_scaler_aut.transform(X_benign)

X_anomalous_dnn = std_scaler_dnn.transform(X_anomalous)
X_anomalous_aut = std_scaler_aut.transform(X_anomalous)

In [35]:
def checkFalseAnomalous(preds_dnn, preds_aut):
    X_benign_with_preds = X_benign.assign(preds_dnn=preds_dnn.astype(bool), preds_aut=np.array(preds_aut).astype(bool))
    X_false_anomalous = X_benign_with_preds[(X_benign_with_preds['preds_dnn'] == True) | (X_benign_with_preds['preds_aut'] == True)]   

    print(f'False anomalous: {X_false_anomalous.shape[0]} / {X_benign.shape[0]} - Rate {(X_false_anomalous.shape[0] / X_benign.shape[0]) * 100}%')

def checkFalseBenign(preds_dnn, preds_aut):
    X_anomalous_with_preds = X_anomalous.assign(preds_dnn=preds_dnn.astype(bool), preds_aut=np.array(preds_aut).astype(bool))
    X_false_benign = X_anomalous_with_preds[(X_anomalous_with_preds['preds_dnn'] == False) & (X_anomalous_with_preds['preds_aut'] == False)]   

    print(f'False benign: {X_false_benign.shape[0]} / {X_anomalous.shape[0]} - Rate {(X_false_benign.shape[0] / X_anomalous.shape[0]) * 100}%')

In [36]:
#non retrained predictions on NON adveersarial samples
preds_benign_dnn = ModelUtils.binary_preds_supervised(dnn, X_benign_dnn)
preds_benign_aut = ModelUtils.binary_preds_unsupervised(autoencoder, X_benign_aut)
preds_anomalous_dnn = ModelUtils.binary_preds_supervised(dnn, X_anomalous_dnn)
preds_anomalous_aut = ModelUtils.binary_preds_unsupervised(autoencoder, X_anomalous_aut)

#retrained prediction on NON adveersarial samples
preds_benign_dnn_retrained = ModelUtils.binary_preds_supervised(dnn_retrained, X_benign_dnn)
preds_benign_aut_retrained = ModelUtils.binary_preds_unsupervised(autoencoder, X_benign_aut, 0.035)
preds_anomalous_dnn_retrained = ModelUtils.binary_preds_supervised(dnn_retrained, X_anomalous_dnn)
preds_anomalous_aut_retrained = ModelUtils.binary_preds_unsupervised(autoencoder, X_anomalous_aut, 0.035)

In [37]:
print('PREDICTION ERROR ON BENIGN SAMPLES -> BASE TARGET SYSTEM:')
checkFalseAnomalous(preds_benign_dnn, preds_benign_aut)
print('----------')
print('PREDICTION ERROR ON BENIGN SAMPLES -> TARGET SYSTEM WITH ADVERSARIAL RETRAINING:')
checkFalseAnomalous(preds_benign_dnn_retrained, preds_benign_aut_retrained)
print('\n')
print('PREDICTION ERROR ON ANOMALOUS SAMPLES -> BASE TARGET SYSTEM:')
checkFalseBenign(preds_anomalous_dnn, preds_anomalous_aut)
print('----------')
print('PREDICTION ERROR ON ANOMALOUS SAMPLES -> TARGET SYSTEM WITH ADVERSARIAL RETRAINING:')
checkFalseBenign(preds_anomalous_dnn_retrained, preds_anomalous_aut_retrained)

PREDICTION ERROR ON BENIGN SAMPLES -> BASE TARGET SYSTEM:
False anomalous: 5691 / 100000 - Rate 5.691%
----------
PREDICTION ERROR ON BENIGN SAMPLES -> TARGET SYSTEM WITH ADVERSARIAL RETRAINING:
False anomalous: 8031 / 100000 - Rate 8.031%


PREDICTION ERROR ON ANOMALOUS SAMPLES -> BASE TARGET SYSTEM:
False benign: 1512 / 100000 - Rate 1.512%
----------
PREDICTION ERROR ON ANOMALOUS SAMPLES -> TARGET SYSTEM WITH ADVERSARIAL RETRAINING:
False benign: 57 / 100000 - Rate 0.056999999999999995%
